### Programming for Biomedical Informatics
#### Week 6 - Differential Gene Expression Analysis

We're going to perform some differential expression analysis using the PyDESeq2 package using an RNA-Seq dataset from NCBI-GEO

In [ ]:
'''
Sources of Data

Original Publication
Tomaiuolo P, Piras IS, Sain SB, Picinelli C, Baccarin M, Castronovo P, Morelli MJ, Lazarevic D, Scattoni ML, Tonon G, Persico AM.
RNA sequencing of blood from sex- and age-matched discordant siblings supports immune and transcriptional dysregulation in autism spectrum disorder.
Sci Rep. 2023 Jan 16;13(1):807. doi: 10.1038/s41598-023-27378-w. PMID: 36646776; PMCID: PMC9842630.

GEO Entry: GSE212645
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE212645

meta-data file
https://ftp.ncbi.nlm.nih.gov/geo/series/GSE212nnn/GSE212645/matrix/GSE212645_series_matrix.txt.gz

raw data file
https://www.ncbi.nlm.nih.gov/geo/download/?type=rnaseq_counts&acc=GSE212645&format=file&file=GSE212645_raw_counts_GRCh38.p13_NCBI.tsv.gz

normalised data file
https://www.ncbi.nlm.nih.gov/geo/download/?type=rnaseq_counts&acc=GSE212645&format=file&file=GSE212645_norm_counts_FPKM_GRCh38.p13_NCBI.tsv.gz

genome annotation file
https://www.ncbi.nlm.nih.gov/geo/download/?format=file&type=rnaseq_counts&file=Human.GRCh38.p13.annot.tsv.gz
'''

In [ ]:
#read in the meta-data
# we need to skip the first 38 rows as they contain project rather than sample meta-data
import pandas as pd
metadata = pd.read_csv('./data/GSE212645_series_matrix.txt.gz', sep='\t', skiprows=38, header=None)

# we now tidy this up and retain the information we need
# keep the rows we need
row_numbers = [0,8,10,11,12]
metadata = metadata.iloc[row_numbers]

# replace column 0 values with the list below
new_feature_names = ['number','gender', 'status', 'family', 'treatment']
metadata.iloc[:,0] = new_feature_names

# make the first row the column names and remove the first row
metadata.columns = metadata.iloc[0]
metadata = metadata.iloc[1:]
metadata.set_index('number', append=False, inplace=True)

# # transpose the data frame
metadata = metadata.T

# # reset the index and rename the first column
metadata.reset_index(inplace=True)
metadata.rename(columns={0: 'sample_no'}, inplace=True)

# tidy up the column contents
metadata['gender'] = metadata['gender'].str.replace('Sex: ', '')
metadata['status'] = metadata['status'].str.replace('genotype: ', '')
metadata['family'] = metadata['family'].str.replace('family: ', '')
metadata['treatment'] = metadata['treatment'].str.replace('treatment: ', '')

metadata.set_index('sample_no', inplace=True)
metadata.index.name = None

#change index name to sample_id
metadata.index.name = 'sample_id'

metadata.head()

In [43]:
# retrive the raw data file

raw_data_url = 'https://www.ncbi.nlm.nih.gov/geo/download/?type=rnaseq_counts&acc=GSE212645&format=file&file=GSE212645_raw_counts_GRCh38.p13_NCBI.tsv.gz'

# download the file and save in the ./data directory
import urllib.request
import os

urllib.request.urlretrieve(raw_data_url, './data/GSE212645_raw_counts_GRCh38.p13_NCBI.tsv.gz')

#read directly into a data frame
import pandas as pd

raw_counts = pd.read_csv('./data/GSE212645_raw_counts_GRCh38.p13_NCBI.tsv.gz', sep='\t', index_col=0)

In [ ]:
raw_counts.head()

In [ ]:
# plot a boxplot of the raw counts by sample
import matplotlib.pyplot as plt

exp_data = raw_counts.copy()

# convert all numbers less than or eual to 10 to nan
import numpy as np
exp_data[exp_data <= 10] = np.nan

plt.figure(figsize=(10,6))

exp_data.boxplot(rot=90)
plt.yscale('log')
plt.title('Raw Gene Counts by Sample <=10 values removed')
plt.ylabel('log(Raw Counts)')
plt.show()

In [ ]:
import numpy as np

# plot an MA plot of the raw counts from scratch

# find all the columns in raw_counts that have column headers matching the metadata index and have staus of 'ASD'ArithmeticError
asd_samples = metadata[metadata['status'] == 'ASD'].index

# find all the columns in raw_counts that have column headers matching the metadata index and have staus of 'SIB'
sib_samples = metadata[metadata['status'] == 'SIB'].index

# calculate the mean of the raw counts for each gene in the ASD samples
asd_mean = raw_counts[asd_samples].mean(axis=1)

# calculate the mean of the raw counts for each gene in the SIB samples
sib_mean = raw_counts[sib_samples].mean(axis=1)

# create a new data frame with these mean values and the gene names as the index
ma_data = pd.DataFrame({'ASD': asd_mean, 'SIB': sib_mean})

# calculate the M value with log2
ma_data['M'] = ma_data['ASD'].apply(np.log2) - ma_data['SIB'].apply(np.log2)

# calculate the A value with log2
ma_data['A'] = (ma_data['ASD'].apply(np.log2) + ma_data['SIB'].apply(np.log2)) / 2

# plot the data
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(ma_data['A'], ma_data['M'], s=1)
plt.xlabel('A')
plt.ylabel('M')
plt.title('MA Plot of Raw Counts')
plt.show()

In [ ]:
# and for the cleaned up test
# calculate the mean of the raw counts for each gene in the ASD samples
asd_mean = exp_data[asd_samples].mean(axis=1)

# calculate the mean of the raw counts for each gene in the SIB samples
sib_mean = exp_data[sib_samples].mean(axis=1)

# create a new data frame with these mean values and the gene names as the index
ma_data = pd.DataFrame({'ASD': asd_mean, 'SIB': sib_mean})

# calculate the M value with log2
ma_data['M'] = ma_data['ASD'].apply(np.log2) - ma_data['SIB'].apply(np.log2)

# calculate the A value with log2
ma_data['A'] = (ma_data['ASD'].apply(np.log2) + ma_data['SIB'].apply(np.log2)) / 2

# plot the data
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(ma_data['A'], ma_data['M'], s=1)
plt.xlabel('A')
plt.ylabel('M')
plt.title('MA Plot of Raw Counts with <=10 values removed')
plt.show()

In [48]:
# now lets use DESeq2 to perform differential expression

import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import *

SAVE = False  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be
    # saved
    OUTPUT_PATH = "./data/asd_deseq2_results"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

In [ ]:
# we need the counts matrix transposed for DESeq2
# samples as rows and genes as columns
raw_counts = raw_counts.T

# keep genes with at least 10 counts in at least 2 samples
raw_counts = raw_counts[(raw_counts > 10).sum(axis=1) >= 2]

# change 'GeneID' to 'sample_id'
raw_counts.index.name = 'sample_id'

raw_counts.head()

In [ ]:
## set up the DESeq2 object
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=raw_counts,
    metadata=metadata,
    design_factors="status",
    refit_cooks=True,
    inference=inference,
)

In [ ]:
# compute normalisation factors
dds.fit_size_factors()

dds.obsm["size_factors"]

In [ ]:
# fit gene-wise dispersion estimates
dds.fit_genewise_dispersions()

gene_dispersion = dds.varm["genewise_dispersions"]

#plot the dispersion estimates
plt.figure(figsize=(10,6))
plt.hist(gene_dispersion, bins=100)
plt.xlabel('Dispersion')
plt.ylabel('Frequency')
plt.title('Gene-wise Dispersion Estimates')
plt.show()

In [ ]:
# fit dispersion trend coefficients
dds.fit_dispersion_trend()

dds.uns["trend_coeffs"]

fitted_dispersions = dds.varm["fitted_dispersions"]

# plot the fitted dispersion trend
plt.figure(figsize=(10,6))
plt.scatter(gene_dispersion, fitted_dispersions)
plt.xlabel('Gene-wise Dispersion')
plt.ylabel('Fitted Dispersion')
plt.title('Fitted Dispersion vs Gene-wise Dispersion')
plt.show()

In [ ]:
# fit dispersion priors
dds.fit_dispersion_prior()
print(f"logres_prior={dds.uns['_squared_logres']},sigma_prior={dds.uns['prior_disp_var']}")

In [ ]:
# fit MAP dispersions
dds.fit_MAP_dispersions()
dds.varm["MAP_dispersions"]
dds.varm["dispersions"]

In [ ]:
# fit log fold changes
dds.fit_LFC()
lfcs = dds.varm["LFC"]

# plot the log fold changes
plt.figure(figsize=(10,6))
plt.hist(lfcs, bins=100)
plt.xlabel('Log Fold Change')
plt.ylabel('Frequency')
plt.title('Log Fold Changes')
plt.show()

In [ ]:
# calculate cooks distanes and refit

# this step aims to identify outliers that would adversely affect the differential expresison analysis and filters them out
dds.calculate_cooks()
if dds.refit_cooks:
    # Replace outlier counts
    dds.refit()

In [ ]:
#let's look at the dispersion plot
dds.plot_dispersions()

In [29]:
# save the results so far
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(dds, f)

In [ ]:
# Statistical Analysis

In [59]:
# this is the main step where the differential expression is calculated
# here we can set our alpha value to 0.05, and also filter out genes with high cook's distance
stat_res = DeseqStats(dds, alpha=0.05, cooks_filter=True, independent_filter=True)

In [ ]:
# run the Wald test
# this test effectively calculates the robustness of the beta value estaimation in the main DESeq2 GLM and then calculates the p-values based on the aussume
# assumed normal distribution of the beta values
stat_res.run_wald_test()
stat_res.p_values

In [ ]:
# filter these using the cook's distance
if stat_res.cooks_filter:
    stat_res._cooks_filtering()
stat_res.p_values

In [ ]:
# now use the independent filtering to reduce the number of tests and hence reduces the multiple testing burden
if stat_res.independent_filter:
    stat_res._independent_filtering()
else:
    stat_res._p_value_adjustment()

stat_res.padj.sort_values(ascending=True)

In [ ]:
stat_res.summary()

In [ ]:
results = stat_res.results_df

sorted_results = results.sort_values(by='pvalue', ascending=True)

#convert the GeneID column to integers
sorted_results.index = sorted_results.index.map(int)
sorted_results.reset_index(inplace=True)

sorted_results.head()

# it's clear here that the p-value adjustment is very heavily penalising the p-values

In [111]:
# optionally save the results
# if SAVE:
#     with open(os.path.join(OUTPUT_PATH, "stat_results_detailed_pipe.pkl"), "wb") as f:
#         pkl.dump(stat_res, f)

In [ ]:
# load up the genome annotation file so that we can look at the gene names

annotation_url = 'https://www.ncbi.nlm.nih.gov/geo/download/?format=file&type=rnaseq_counts&file=Human.GRCh38.p13.annot.tsv.gz'

# download the file and save in the ./data directory
import urllib.request
import os

urllib.request.urlretrieve(annotation_url, './data/Human.GRCh38.p13.annot.tsv.gz')

#read directly into a data frame
annotation = pd.read_csv('./data/Human.GRCh38.p13.annot.tsv.gz', sep='\t', index_col=0, low_memory=False)

#drop all columns except Symbol and Description
annotation = annotation[['Symbol', 'Description']]

annotation.head()

In [ ]:
# merge the annotation with the results on the GeneID column
results = sorted_results.merge(annotation, left_on='GeneID', right_on='GeneID')

results.head(40)

In [ ]:
# now lets go back and pick up the results for the genes cited in the paper to see what the p-values are
target_genes = ['EGR1', 'EGR2', 'IGKV6D-21', 'IGKV3D-15', 'S100B', 'CD80']

target_gene_results = results[results['Symbol'].isin(target_genes)]

target_gene_results.head()

In [ ]:
#lets go back and find the log normlised counts for these genes
# merge the ma_data with the target_gene_results on the GeneID column

combined = target_gene_results.merge(ma_data, left_on='GeneID', right_index=True)

combined.head(10)

In [ ]:
# next we plot paired barcharts for each Symbol showing the log normalised counts for each gene in the ASD and SIB samples using matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 6, figsize=(20, 6))

for i, gene in enumerate(target_genes):
    #take the log2 values of the counts
    combined.loc[combined['Symbol'] == gene, ['ASD', 'SIB']] = combined.loc[combined['Symbol'] == gene, ['ASD', 'SIB']].apply(np.log2)
    ax[i].bar(['ASD', 'SIB'], combined.loc[combined['Symbol'] == gene, ['ASD', 'SIB']].values[0])
    ax[i].set_title(gene)
    ax[i].set_ylabel('log normalised counts')
    ax[i].set_xlabel('Status')

plt.show()

# the trends are the same as in the paper, but the p-values are not significant - NB in the paper only EGR1 and IGKV3D-15 were significant after correction.